In [22]:
import pandas as pd
df = pd.read_csv("clean_test_or_not_test.csv", index_col=0)

In [23]:
df.head(-6)

,text_clean,label
0,cover anvil service response in unit testshttp...,0
1,service apprepositorycontroller should be cach...,0
2,many caches persist between test casesan examp...,0
3,improve friendliness of behat text pattern mat...,0
4,run e2e tests for prs is your feature request ...,0
...,...,...
431,add multithreaded command to skuldthey need to...,1
432,updated getparcableextra deprecated methodremo...,1
433,migrate componentssuggestionproviderjsx to typ...,1
434,support admin device form validationcurrently ...,1


In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming df_filtered contains the filtered dataset with label=1
texts = df['text_clean'].values
labels = df['label'].values

# Tokenize the texts
tokenizer = Tokenizer(num_words=5000)  # Adjust `num_words` as needed
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences
max_sequence_length = max(len(x) for x in sequences)  # Or choose a fixed length
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to a numpy array
y = np.array(labels)


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

embedding_dim = 100  # Size of the embedding vector

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_dim))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(max_sequence_length, embedding_dim)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/itf-fi-ml/home/nehad/.conda/envs/dl/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 0.5344 - loss: 0.6904 - val_accuracy: 0.4607 - val_loss: 0.6871
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.8677 - loss: 0.6374 - val_accuracy: 0.7528 - val_loss: 0.6634
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.9631 - loss: 0.5987 - val_accuracy: 0.7416 - val_loss: 0.6383
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.9527 - loss: 0.5474 - val_accuracy: 0.8315 - val_loss: 0.6062
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.9664 - loss: 0.4902 - val_accuracy: 0.8315 - val_loss: 0.5611


In [27]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

import time

# Start the timer
start_time = time.time()

predictions = model.predict(X_test)
# Convert predictions to binary outcomes
y_pred = (predictions > 0.5).astype(int).flatten()

# End the timer
end_time = time.time()
runtime = end_time - start_time


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step


In [28]:
from sklearn.metrics import accuracy_score

# y_test: true labels
# y_pred: predicted labels
accuracy = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='binary')  # Use 'macro' or 'weighted' for multi-class
precision = precision_score(y_test, y_pred, average='binary')  # Adjust as needed
recall = recall_score(y_test, y_pred, average='binary')  # Adjust as needed


In [29]:
sample_percentage = (len(X_test) / (len(X_test) + len(X_train))) * 100


In [30]:

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Runtime for predictions: {runtime} seconds')
print(f'Sample percentage of test set: {sample_percentage:.2f}%')


Accuracy: 0.8314606741573034
F1 Score: 0.823529411764706
Precision: 0.7777777777777778
Recall: 0.875
Runtime for predictions: 0.8328125476837158 seconds
Sample percentage of test set: 20.14%


In [31]:
from tensorflow.keras.models import load_model

model.save('my_model.h5')

# Or, loading from the HDF5 format
model = load_model('my_model.h5')
